<a href="https://colab.research.google.com/github/amb467/cs2770_hw3/blob/master/CS2770_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run when starting a new runtime
This part does the following:

1.   Clones my GitHub repository for this homework assignment
2.   Installs NLTK punkt, which is used for word tokenization
3.   Downloads and unzips the COCO images.  To save space, it deletes the zip file once it's unzipped.
4.   Prepares the data needed to create the COCO data loader.  This is serialized and saved as /content/cs2770_hw3/data/coco.pkl



In [ ]:
!git clone https://github.com/amb467/cs2770_hw3.git
import nltk
nltk.download('punkt')
!wget http://images.cocodataset.org/zips/val2014.zip
!unzip /content/val2014.zip
!rm /content/val2014.zip
!python3 /content/cs2770_hw3/data_loader.py --output_dir /content/cs2770_hw3/data --image_dir /content/val2014 --coco_data_file /content/cs2770_hw3/captions/captions_val2014.json

# Prepare the word embeddings
My baseline word embeddings are **50-dimensional GloVe** and I use **Word2Vec pre-trained on the Good News corpus** as my alternate word representation for Part C.  The Word2Vec embeddings start out as 300-dimensional but I use Principal Component Analysis to reduce them to 50 dimensions for parity with the GloVe embeddings.  I normalize both sets of embeddings.

Once the word embeddings are loaded into dictionary objects, they are serialized as glove.pkl and word2vec.pkl in the /content/cs2770_hw3/data/ directory

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip /content/glove.6B.zip
!rm /content/glove.6B.zip
!rm /content/glove.6B.100d.txt
!rm /content/glove.6B.200d.txt
!rm /content/glove.6B.300d.txt
!python3 /content/cs2770_hw3/data_loader.py --glove_embedding /content/glove.6B.50d.txt --output_dir /content/cs2770_hw3/data
!rm /content/glove.6B.50d.txt

# Prepare Good News image data set
Note that this takes a while (25-30 minutes) so only run this if you need the 
Good News set.  I use this as my alternate data set for Part B.

All of the Good News images are downloaded to /content/good_news .  As with the COCO data set, the data needed to create a data loader is serialized and saved in the /content/cs2770_hw3/data folder.

If the images are already downloaded, save a lot of time by removing the --news_download_images flag

In [ ]:
!python3 /content/cs2770_hw3/data_loader.py --news_download_images --news_data_file /content/cs2770_hw3/good_news.txt --image_dir /content/good_news --output_dir /content/cs2770_hw3/data  

# Run all HW3 parts except for the cross-domain adaptation from Part B
The script is set up to run the defaults, which are:

## Defaults
*   **--epochs** - this defaults to 25, which was used for all models described in the results below
*   **--image_data_set** - this defaults to "coco" but can also be run with "news" for the Good News corpus
*   **--embedding** - this defaults to "glove" but can also be run with "w2v" to use the Word2Vec embeddings
*   **--model** - this defaults to "alex" for AlexNet but can also be run with "res" for ResNet18

## Other parameters:
*   **--data_dir** - this is the directory where serialized objects for image data sets and word embeddings can be found.  It should be left as /content/cs2770_hw3/data
*   **--output_dir** - this is the directory where the models will be saved
*   **--cross_domain_eval** - when this flag is included, the results will additionally include results from running the model on the alternate data set.  The alternate data set will be the Good News test set if the model was trained on COCO and vice versa.

## Homework Configurations
### Part A

```
python3 /content/cs2770_hw3/hw3.py --data_dir /content/cs2770_hw3/data --output_dir /content/cs2770_hw3/output
```

### Part B
Note that this is just for cross-domain evaluation. cross-domain adaptation is run with a different script below.

```
python3 /content/cs2770_hw3/hw3.py --cross_domain_eval --data_dir /content/cs2770_hw3/data --output_dir /content/cs2770_hw3/output
```

### Part C
To run with ResNet18 instead of AlexNet:

```
python3 /content/cs2770_hw3/hw3.py --model res --data_dir /content/cs2770_hw3/data --output_dir /content/cs2770_hw3/output
```

To run with Word2Vec embeddings instead of GloVe:

```
python3 /content/cs2770_hw3/hw3.py --embedding w2v --data_dir /content/cs2770_hw3/data --output_dir /content/cs2770_hw3/output
```


In [ ]:
!python3 /content/cs2770_hw3/hw3.py --data_dir /content/cs2770_hw3/data --output_dir /content/cs2770_hw3/output

# Cross-Domain Adaptation
This script is similar to the one described above but it does not allow all of the variations.  The only parameters are --epochs, --data_dir, and --output_dir .  The script uses AlexNet as the model, GloVe as the embeddings, COCO as the source data set and Good News as the target data set.

As with the previous script, the default number of epochs is 25 and this was used for the results shown below.

In [ ]:
!python /content/cs2770_hw3/hw3_dom_adaptation.py --data_dir /content/cs2770_hw3/data --output_dir /content/cs2770_hw3/output

# Results
---
## Part A
* Model: AlexNet
* Image data set: COCO
* Embeddings: GloVe
* Epochs: 25

| Metric | Accuracy | % Difference from Random |
| --- | --- | --- |
| Text to image | 0.0375 | 380.15% |
| Image to text | 0.0150 | 92.06% |

## Part B
### Cross-Domain Evaluation
* Model: AlexNet
* Image data set (training): COCO
* Image data set (testing): Good News
* Embeddings: GloVe
* Epochs: 25

| Metric | Accuracy | % Difference from Random |
| --- | --- | --- |
| Text to image | 0.0145 | 85.66% |
| Image to text | 0.0091 | 16.52% |

### Cross-Domain Adaptation

* Model: AlexNet
* Image data set (source): COCO
* Image data set (target): Good News
* Image data set (testing): COCO
* Embeddings: GloVe
* Epochs: 25

| Metric | Accuracy | % Difference from Random |
| --- | --- | --- |
| Text to image | 0.01 | 28.04% |
| Image to text | 0.01 | 28.04% |

* Model: AlexNet
* Image data set (source): COCO
* Image data set (target): Good News
* Image data set (testing): Good News
* Embeddings: GloVe
* Epochs: 25

| Metric | Accuracy | % Difference from Random |
| --- | --- | --- |
| Text to image | 0.01 | 28.04% |
| Image to text | 0.01 | 28.04% |

## Part C
### ResNet18 instead of AlexNet
* Model: Resnet18
* Image data set: COCO
* Embeddings: GloVe
* Epochs: 25

| Metric | Accuracy | % Difference from Random |
| --- | --- | --- |
| Text to image | 0.0375 | 380.15% |
| Image to text | 0.0075 | -3.97% |

### Word2Vec instead of GloVe
* Model: AlexNet
* Image data set: Word2Vec
* Embeddings: GloVe
* Epochs: 25

| Metric | Accuracy | % Difference from Random |
| --- | --- | --- |
| Text to image | 0.0475 | 508.19% |
| Image to text | 0.0125 | 60.05% |

---
## Analysis

The batch size for all models was 128 and retrieval was formulated as a top-1 accuracy problem of (for image to text) selecting the correct caption for an image out of 128 captions and (for text to image) selecting the correct image for a caption out of 128 images.  This means that the likelihood of selecting the correct option by chance is 0.00781 or less than 0.8%.  The accuracy for our models generally does not look very high, but when compared to a random baseline we see that the models mostly do quite a bit better than random.

In all cases, the image-to-text accuracy is much higher than than text-to-image accuracy.  My hunch is that this is related to how information is combined to form the 50-dimensional vector representation of each object.  Both AlexNet and ResNet are architected to try to capture what is most important about the image, and they use a number of filters to try to activate the most important elements of the image.  This is also true of the statisical models that trained both word embeddings, GloVe and Word2Vec, but that is at the word level, not the span level.  When combining all of the words in each description, I simply averaged the embedding of each word.  I did not perform any operation to give greater weight to more important or salient words while de-emphasizing less important words such as "the" and "and".  I did not remove stopwords.  Therefore, the images may be better differentiated in the trained vector space, making it easier to select the closest one to a particular text description.  The text descriptions may end up more clumped in the vector space, making it easier to select the wrong one.

Looking at cross-domain evaluation (without cross-domain adaptation), the performance of both image-to-text and text-to-image when testing on the Good News corpus was worse than when testing on the COCO test set.  This is to be expected because the images in the COCO test set are more similar to the images that were used for training than the Good News images.

The numbers for cross-domain adaptation are bizarre: all of the values are 0.01.  This can't be a problem with the test function because my cross-domain adaptation models use the same test method as the other models (hw3_dom_adaptation.py actually imports this function from hw3.py, so it's literally the same function).  The performance is still quite a bit better than random, but clearly something has gone wrong here.

Comparing the performance of AlexNet and ResNet18, the text-to-image performance is exactly the same, though the image-to-text performance is quite a bit worse with ResNet18.  I can't really imagine why this is the case and I suspect it might just be statistical noise.  Following my theory about why the text-to-image numbers are so much better than image-to-text, if if were the text-to-image numbers that were significantly worse with ResNet18 then I would say that perhaps the image vectors are not as rich and differentiated as with AlexNet, leading to poorer performance.

Comparing the performance of the default model using GloVe embeddings compared to the model that uses Word2Vec, the image-to-text performance is quite a bit better, while the text-to-image is worse.  I'm not sure why the image-to-text numbers are better, but the text-to-image performance may be related to the fact that I used Principal Component Analysis to reduce the 300-dimensional Word2Vec embeddings to 50 dimensions so that they had the same dimensions as the 50-dimensional GloVe embeddings.  It's possible that that this process reduced the efficacy of the Word2Vec embeddings.  Another theory is that the GloVe embeddings are simply better semantic representations of the words than the Word2Vec embeddings.

Future steps to improve these models might include:

* Investigating whatever went wrong with the cross-domain adaptation.
* Using an RNN to train span-level embeddings that better represent the semantic meaning of each image description rather than simply averaging over word-level embeddings.
* Playing with hyperparameters to better improve overall performance.






## Other Stuff
This is a code block to pull updates from the git repo followed by some handy debugging code to display an image.

In [ ]:
!git -C /content/cs2770_hw3 pull https://github.com/amb467/cs2770_hw3.git 

In [ ]:
from PIL import Image
from matplotlib.pyplot import imshow

img_path = '/content/good_news/58f7352c95d0e024b586c1c3_0.jpg'
image = Image.open(img_path).convert('RGB')
imshow(image)